In [6]:
from process import *
from model import *
import pandas as pd
%load_ext autoreload
%autoreload 2

In [16]:
# if processing the entire training set, this should take a while
post = get_post('train')
post.head()

,TripType,VisitNumber,Upc,ScanCount,FinelineNumber,Friday,Monday,Saturday,Sunday,Thursday,...,containsReturn,mostFreqFineLine,totalItemsBought,totalDistinctItemsBought,itemDist,numTransactions,totalReturns,numUniqueFinelines,avePurchaseSize,medianPurchaseSize
0,999,5,68113152929,-1,1000,1,0,0,0,0,...,1,-1,0,0,0.000000,1,1,1,-1.000000,-1
1,30,7,60538815980,1,8931,1,0,0,0,0,...,0,1,2,2,0.000000,2,0,2,1.000000,1
2,30,7,7410811099,1,4504,1,0,0,0,0,...,0,1,2,2,0.000000,2,0,2,1.000000,1
3,26,8,2238403510,2,3565,1,0,0,0,0,...,1,1,30,21,0.882512,23,2,17,1.217391,1
4,26,8,2006613744,2,1017,1,0,0,0,0,...,1,1,30,21,0.882512,23,2,17,1.217391,1


In [57]:
rf = get_forest()

In [8]:
sub = prep_for_submission()

AttributeError: 'forest' object has no attribute 'test'